# Система моніторингу розташування підрозділів

Цей ноутбук містить функції для роботи з базою даних UnitDeploymentDB, яка зберігає інформацію про військову техніку та її розташування.

## Імпорт необхідних бібліотек

In [1]:
import sqlite3
import pandas as pd
import json
from flask import Flask, request, jsonify
from IPython.display import display, HTML

## Підключення до бази даних

Створюємо підключення до бази даних UnitDeploymentDB. Якщо база даних не існує, вона буде створена автоматично.

In [2]:
# Функція для підключення до бази даних
def connect_to_db(db_name='UnitDeploymentDB.db'):
    conn = sqlite3.connect(db_name)
    return conn

# Створення таблиці для зберігання даних про військову техніку, якщо вона не існує
def create_tables():
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Створення таблиці для військової техніки
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS MilitaryEquipment (
        EquipmentID INTEGER PRIMARY KEY,
        Type TEXT NOT NULL,
        Model TEXT NOT NULL,
        Status TEXT NOT NULL,
        Location TEXT,
        Coordinates TEXT,
        LastUpdated TEXT
    )
    ''')
    
    conn.commit()
    conn.close()
    print('Таблиці успішно створено')

# Виконуємо створення таблиць
create_tables()

Таблиці успішно створено


## Додавання тестових даних

Додамо кілька тестових записів до бази даних для демонстрації функціональності.

In [3]:
# Функція для додавання тестових даних
def add_test_data():
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Перевіряємо, чи є вже дані в таблиці
    cursor.execute('SELECT COUNT(*) FROM MilitaryEquipment')
    count = cursor.fetchone()[0]
    
    if count == 0:
        # Додаємо тестові дані
        test_data = [
            (1, 'Танк', 'T-64БВ', 'Активний', 'Харківська область', '{"lat": 49.9935, "lng": 36.2304}', '2023-05-15 10:30:00'),
            (2, 'БТР', 'БТР-4', 'Ремонт', 'Київська область', '{"lat": 50.4501, "lng": 30.5234}', '2023-05-14 08:45:00'),
            (3, 'САУ', '2С3 Акація', 'Активний', 'Донецька область', '{"lat": 48.0159, "lng": 37.8028}', '2023-05-15 12:15:00'),
            (4, 'Танк', 'T-72', 'Неактивний', 'Львівська область', '{"lat": 49.8397, "lng": 24.0297}', '2023-05-13 16:20:00'),
            (5, 'БМП', 'БМП-2', 'Активний', 'Одеська область', '{"lat": 46.4825, "lng": 30.7233}', '2023-05-15 09:10:00')
        ]
        
        cursor.executemany('INSERT INTO MilitaryEquipment VALUES (?, ?, ?, ?, ?, ?, ?)', test_data)
        conn.commit()
        print(f'Додано {len(test_data)} тестових записів')
    else:
        print(f'Таблиця вже містить {count} записів. Тестові дані не додано.')
    
    conn.close()

# Додаємо тестові дані
add_test_data()

Додано 5 тестових записів


## Функція для отримання всіх записів про військову техніку

In [4]:
def get_all_equipment():
    conn = connect_to_db()
    
    # Використовуємо pandas для зручного відображення даних
    df = pd.read_sql_query('SELECT * FROM MilitaryEquipment', conn)
    
    conn.close()
    return df

# Отримуємо та відображаємо всі записи
all_equipment = get_all_equipment()
display(all_equipment)

,EquipmentID,Type,Model,Status,Location,Coordinates,LastUpdated
0,1,Танк,T-64БВ,Активний,Харківська область,"{""lat"": 49.9935, ""lng"": 36.2304}",2023-05-15 10:30:00
1,2,БТР,БТР-4,Ремонт,Київська область,"{""lat"": 50.4501, ""lng"": 30.5234}",2023-05-14 08:45:00
2,3,САУ,2С3 Акація,Активний,Донецька область,"{""lat"": 48.0159, ""lng"": 37.8028}",2023-05-15 12:15:00
3,4,Танк,T-72,Неактивний,Львівська область,"{""lat"": 49.8397, ""lng"": 24.0297}",2023-05-13 16:20:00
4,5,БМП,БМП-2,Активний,Одеська область,"{""lat"": 46.4825, ""lng"": 30.7233}",2023-05-15 09:10:00


## Функція для пошуку техніки за полем Status

In [5]:
def find_equipment_by_status(status):
    conn = connect_to_db()
    
    # Використовуємо параметризований запит для безпеки
    query = 'SELECT * FROM MilitaryEquipment WHERE Status = ?'
    df = pd.read_sql_query(query, conn, params=(status,))
    
    conn.close()
    return df

# Приклад використання: пошук активної техніки
active_equipment = find_equipment_by_status('Активний')
print(f'Знайдено {len(active_equipment)} одиниць активної техніки:')
display(active_equipment)

Знайдено 3 одиниць активної техніки:


,EquipmentID,Type,Model,Status,Location,Coordinates,LastUpdated
0,1,Танк,T-64БВ,Активний,Харківська область,"{""lat"": 49.9935, ""lng"": 36.2304}",2023-05-15 10:30:00
1,3,САУ,2С3 Акація,Активний,Донецька область,"{""lat"": 48.0159, ""lng"": 37.8028}",2023-05-15 12:15:00
2,5,БМП,БМП-2,Активний,Одеська область,"{""lat"": 46.4825, ""lng"": 30.7233}",2023-05-15 09:10:00


## REST API для оновлення статусу техніки за EquipmentID

In [6]:
# Створюємо Flask додаток для REST API
app = Flask(__name__)

@app.route('/api/equipment/<int:equipment_id>', methods=['PUT'])
def update_equipment_status(equipment_id):
    # Отримуємо дані з запиту
    data = request.get_json()
    
    if not data or 'status' not in data:
        return jsonify({'error': 'Необхідно вказати новий статус в полі "status"'}), 400
    
    new_status = data['status']
    
    # Підключаємося до бази даних
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Перевіряємо, чи існує техніка з вказаним ID
    cursor.execute('SELECT * FROM MilitaryEquipment WHERE EquipmentID = ?', (equipment_id,))
    equipment = cursor.fetchone()
    
    if not equipment:
        conn.close()
        return jsonify({'error': f'Техніку з ID {equipment_id} не знайдено'}), 404
    
    # Оновлюємо статус
    cursor.execute(
        'UPDATE MilitaryEquipment SET Status = ?, LastUpdated = datetime("now") WHERE EquipmentID = ?',
        (new_status, equipment_id)
    )
    
    conn.commit()
    conn.close()
    
    return jsonify({
        'success': True,
        'message': f'Статус техніки з ID {equipment_id} оновлено на "{new_status}"'
    })

# Функція для запуску API сервера
def run_api_server():
    app.run(debug=True, port=5000)
    
# Примітка: щоб запустити API сервер, виконайте наступний код в окремій комірці:
# run_api_server()

# Приклад запиту для оновлення статусу (для демонстрації):
print("Приклад curl-запиту для оновлення статусу техніки:")
print("curl -X PUT http://localhost:5000/api/equipment/1 -H \"Content-Type: application/json\" -d \"{\"status\":\"Ремонт\"}\"")

# Для тестування в Jupyter можна використати наступну функцію
def test_update_status(equipment_id, new_status):
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Перевіряємо поточний статус
    cursor.execute('SELECT Status FROM MilitaryEquipment WHERE EquipmentID = ?', (equipment_id,))
    current = cursor.fetchone()
    
    if not current:
        print(f'Техніку з ID {equipment_id} не знайдено')
        conn.close()
        return
    
    print(f'Поточний статус техніки з ID {equipment_id}: {current[0]}')
    
    # Оновлюємо статус
    cursor.execute(
        'UPDATE MilitaryEquipment SET Status = ?, LastUpdated = datetime("now") WHERE EquipmentID = ?',
        (new_status, equipment_id)
    )
    
    conn.commit()
    
    # Перевіряємо оновлений статус
    cursor.execute('SELECT Status FROM MilitaryEquipment WHERE EquipmentID = ?', (equipment_id,))
    updated = cursor.fetchone()
    
    print(f'Оновлений статус техніки з ID {equipment_id}: {updated[0]}')
    
    conn.close()

# Приклад тестування функції оновлення статусу
# test_update_status(1, 'Ремонт')

Приклад curl-запиту для оновлення статусу техніки:
curl -X PUT http://localhost:5000/api/equipment/1 -H "Content-Type: application/json" -d "{"status":"Ремонт"}"


## Візуалізація розташування техніки на карті

In [8]:
pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/81/4d/bcbee0676dc06f0b014a030cb928e158c96daafb872a01ddee7e60a8c998/folium-0.19.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/f8/9d/91cddd38bd00170aad1a4b198c47b4ed716be45c234e09b835af41f4e717/branca-0.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for xyzservices from https://files.pythonhosted.org/packages/d6/7d/b77455d7c7c51255b2992b429107fab811b2e36ceaf76da1e55a045dc568/xyzservices-2025.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   --------------------------


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Для візуалізації на карті можна використати бібліотеку folium
# Спочатку потрібно встановити її: !pip install folium

try:
    import folium
    
    def visualize_equipment_on_map():
        # Отримуємо дані про техніку
        df = get_all_equipment()
        
        # Створюємо карту, центровану на Україні
        m = folium.Map(location=[49.0, 31.0], zoom_start=6)
        
        # Додаємо маркери для кожної одиниці техніки
        for _, row in df.iterrows():
            try:
                # Парсимо координати з JSON рядка
                coords = json.loads(row['Coordinates'])
                lat, lng = coords['lat'], coords['lng']
                
                # Визначаємо колір маркера залежно від статусу
                if row['Status'] == 'Активний':
                    color = 'green'
                elif row['Status'] == 'Ремонт':
                    color = 'orange'
                elif row['Status'] == 'Неактивний':
                    color = 'red'
                else:
                    color = 'blue'
                
                # Створюємо спливаюче вікно з інформацією
                popup_text = f"<b>{row['Type']} {row['Model']}</b><br>"
                popup_text += f"Статус: {row['Status']}<br>"
                popup_text += f"Розташування: {row['Location']}<br>"
                popup_text += f"Останнє оновлення: {row['LastUpdated']}"
                
                # Додаємо маркер на карту
                folium.Marker(
                    [lat, lng],
                    popup=folium.Popup(popup_text, max_width=300),
                    tooltip=f"{row['Type']} {row['Model']}",
                    icon=folium.Icon(color=color)
                ).add_to(m)
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Помилка при обробці координат для техніки ID {row['EquipmentID']}: {e}")
        
        # Відображаємо карту
        display(m)
    
    # Викликаємо функцію візуалізації
    # visualize_equipment_on_map()
    print("Для візуалізації на карті виконайте команду: visualize_equipment_on_map()")
    
except ImportError:
    print("Для візуалізації на карті потрібно встановити бібліотеку folium: !pip install folium")

Для візуалізації на карті виконайте команду: visualize_equipment_on_map()


In [12]:
visualize_equipment_on_map()

## Висновок

У цьому ноутбуку ми створили систему для моніторингу розташування військової техніки з такими можливостями:

1. Створення та підключення до бази даних UnitDeploymentDB
2. Отримання всіх записів про військову техніку
3. Пошук техніки за полем Status
4. REST API для оновлення статусу техніки за EquipmentID
5. Візуалізація розташування техніки на карті (за наявності бібліотеки folium)

Для запуску REST API сервера виконайте функцію `run_api_server()` в окремій комірці.